In [1]:
import os
import z_function_list
from functools import reduce

In [2]:
base_path,data,outputs = z_function_list.folder_setup()

In [3]:
os.chdir(data)

In [210]:
#pull in excel file
cleanstat = pd.ExcelFile('clean_comparison.xlsx')
#create dictionary of frames for each sheet
d = {n:cleanstat.parse(n) for n in cleanstat.sheet_names}
#merge into single DF
combined_quarters = reduce(lambda x,y: pd.merge(x,y,on='SegmentID'),[d[sheet] for sheet in cleanstat.sheet_names])

#replace columns names to make more specific
combined_quarters.columns = combined_quarters.columns.str.replace('_x','_Q1_2016').str.replace('_y','_Q4_2016')
column_replacement = {n:k for n,k in zip(combined_quarters.columns[-5:],[x+'_Q4_2017' for x  in combined_quarters.columns[-5:]])}
combined_quarters = combined_quarters.rename(columns=column_replacement)

#add in missing CD
missing_cd_map = {k:v for k,v in pd.read_csv('clean_stat_missing_CD.csv')[['SegmentID','District']].values}

In [211]:
combined_quarters.columns

Index(['OBJECTID_1_Q1_2016', 'FullName_Q1_2016', 'LAPD_Grid_Q1_2016',
       'GlobalID_Q1_2016', 'SegmentID', 'xy', 'CD_Q1_2016',
       'CS_RoundSc_Q1_2016', 'Miles_Q1_2016', 'OBJECTID_1_Q4_2016',
       'FullName_Q4_2016', 'LAPD_Grid_Q4_2016', 'GlobalID_Q4_2016',
       'CS_RoundSc_Q4_2016', 'CD_Q4_2016', 'SanDist', 'Miles_Q4_2016',
       'OBJECTID_1_Q4_2017', 'FullName_Q4_2017', 'LAPD_Grid_Q4_2017',
       'CS_RoundSc_Q4_2017', 'Miles_Q4_2017'],
      dtype='object')

In [212]:
combined_quarters = combined_quarters.drop(['FullName_Q1_2016', 'LAPD_Grid_Q1_2016','CD_Q1_2016',
                                            'GlobalID_Q1_2016','GlobalID_Q4_2016','Miles_Q1_2016',
                                            'Miles_Q4_2017','FullName_Q4_2016','LAPD_Grid_Q4_2016',
                                           'OBJECTID_1_Q1_2016', 'OBJECTID_1_Q4_2016',
                                            'OBJECTID_1_Q4_2017'],axis=1)

combined_quarters = combined_quarters.rename(columns={'Miles_Q4_2016':'Miles',
                                                      'LAPD_Grid_Q4_2017':'LAPD_Grid',
                                                     'CD_Q4_2016':'CD',
                                                     'FullName_Q4_2017':'FullName'})

combined_quarters = combined_quarters[['FullName','SegmentID','CD','CS_RoundSc_Q1_2016','CS_RoundSc_Q4_2016',
                   'CS_RoundSc_Q4_2017','Miles','SanDist','LAPD_Grid']]


#updating missing CDs
combined_quarters.loc[combined_quarters['CD']==0,'CD'] = combined_quarters.loc[combined_quarters['CD']==0,'SegmentID'].map(missing_cd_map)

#earlier version to verify true mileage differences
#combined_quarters[['Miles_Q1_2016','Miles_Q4_2016','Miles_Q4_2017']] = combined_quarters[['Miles_Q1_2016','Miles_Q4_2016','Miles_Q4_2017']].round(decimals=5)

street_mapping = {1:'Clean',2:'Somewhat Clean',3:'Not Clean',0:'Not Evaluated'}

combined_quarters['Cat_Q1_2016'],combined_quarters['Cat_Q4_2016'],combined_quarters['Cat_Q4_2017']=\
combined_quarters['CS_RoundSc_Q1_2016'].map(street_mapping),\
combined_quarters['CS_RoundSc_Q4_2016'].map(street_mapping),\
combined_quarters['CS_RoundSc_Q4_2017'].map(street_mapping)

In [213]:
q1_2016 = combined_quarters.pivot_table('Miles',columns='Cat_Q1_2016',index='CD',aggfunc='sum')
q4_2016 = combined_quarters.pivot_table('Miles',columns='Cat_Q4_2016',index='CD',aggfunc='sum')
q4_2017 = combined_quarters.pivot_table('Miles',columns='Cat_Q4_2017',index='CD',aggfunc='sum')

In [201]:
q4_2017.loc[[8,9,10]][['Clean','Somewhat Clean','Not Clean']]

Cat_Q4_2017,Clean,Somewhat Clean,Not Clean
CD,,,
8,317.588071,152.066341,9.378174
9,238.716445,110.356725,15.378267
10,263.350688,101.977406,2.123125


In [216]:
os.chdir(outputs)
combined_quarters.to_csv('cleanstat_Q1Q42016_Q42017.csv')